In [ ]:
# !pip install darts
# !pip install pythresh    

In [1]:
from darts.models import (
    RNNModel,
    TCNModel,
    TransformerModel,
    NBEATSModel,
    BlockRNNModel,
    VARIMA,
)
from darts.metrics import mape, smape, mae
from darts.dataprocessing.transformers import Scaler
from darts.utils.callbacks import TFMProgressBar
import evaluation_utils, data_utils
from darts import TimeSeries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
def generate_torch_kwargs():
    # run torch models on CPU, and disable progress bars for all model stages except training.
    return {
        "pl_trainer_kwargs": {
            "accelerator": "cpu",
            "callbacks": [TFMProgressBar(enable_train_bar_only=True)],
        }
    }

c:\Users\madalina.ciortan\AppData\Local\miniconda3\envs\europython\lib\site-packages\dask\dataframe\__init__.py:31: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [2]:
X, y = data_utils.get_data('data/6_cardio.npz')

In [3]:
n = len(X)
# Create a DatetimeIndex
time = pd.date_range(start='2020-01-01', periods=n, freq='D')

# df = pd.DataFrame(data=X, index=time)

In [4]:
train_size = int(len(X) * 0.7)

In [5]:
X_train, y_train = X[:train_size], y[:train_size]
X_test, y_test = X[train_size:], y[train_size:]

In [6]:
train_time_series_list = [
    TimeSeries.from_times_and_values(time[:train_size], col) for col in X_train.T]
test_time_series_list = [
    TimeSeries.from_times_and_values(time[train_size:], col) for col in X_test.T]

In [7]:
model = NBEATSModel(
    input_chunk_length=1000,
    output_chunk_length=200,
    n_epochs=5,
    random_state=0,
    **generate_torch_kwargs()
)

In [8]:
model.fit(train_time_series_list)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name          | Type             | Params
---------------------------------------------------
0 | criterion     | MSELoss          | 0     
1 | train_metrics | MetricCollection | 0     
2 | val_metrics   | MetricCollection | 0     
3 | stacks        | ModuleList       | 13.9 M
---------------------------------------------------
13.9 M    Trainable params
7.3 K     Non-trainable params
13.9 M    Total params
111.216   Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

c:\Users\madalina.ciortan\AppData\Local\miniconda3\envs\europython\lib\site-packages\torch\__init__.py:749: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at ..\torch\csrc\tensor\python_tensor.cpp:433.)
  _C._set_default_tensor_type(t)
`Trainer.fit` stopped: `max_epochs=5` reached.


NBEATSModel(output_chunk_shift=0, generic_architecture=True, num_stacks=30, num_blocks=1, num_layers=4, layer_widths=256, expansion_coefficient_dim=5, trend_polynomial_degree=2, dropout=0.0, activation=ReLU, input_chunk_length=1000, output_chunk_length=200, n_epochs=5, random_state=0, pl_trainer_kwargs={'accelerator': 'cpu', 'callbacks': [<darts.utils.callbacks.TFMProgressBar object at 0x000002D6DC803490>]})

In [28]:
# Reconstruction
# pred = model.historical_forecasts(train_time_series_list,forecast_horizon = 200, retrain=False)
# pred = np.array([p.values().reshape(-1) for p in pred]).T


In [ ]:
pred = model.predict(n=len(X_test), series=train_time_series_list)
pred = np.array([p.values().reshape(-1) for p in pred]).T

In [ ]:
y_pred = evaluation_utils.threshold_anomalies(X_test, pred, method="filter")
print(evaluation_utils.run_evaluation(
    y_test, np.array(y_pred).reshape(-1),
    do_point_adjustment=True))

In [ ]:
y_pred = evaluation_utils.threshold_anomalies(X_test, pred, method="aucp")
print(evaluation_utils.run_evaluation(
    y_test, np.array(y_pred).reshape(-1),
    do_point_adjustment=True))